# Управление моторами/роботом (Часть 2)

## Управление двумя моторами

Далее мы поговорим о таких важных классах как **MoveSteering** и **MoveTank**.

### *Примечание*

Далее использование сивола __*__ после названий функций (методов) означает, что функции (методы) предоставлены в ознакомительных целях _(либо в текущей комплектации робота не заработает, либо сейчас не требуются)_.

### MoveTank *(left_motor_port, right_motor_port, desc=None, motor_class=<class 'ev3dev2.motor.LargeMotor'>)*

Управляет парой двигателей одновременно, используя индивидуальные уставки скорости для каждого двигателя *(значения left_speed и значения right_speed)*.

#### on_for_degrees *(left_speed, right_speed, degrees, brake=True, block=True)*

Вращаeт двигатели с **левой скоростью** и **правой скоростью** в **градусах**. Скорости могут быть выражены в процентах или в любой реализации значения скорости.

Если левая скорость не равна правой скорости (т.е. робот повернет), двигатель, расположенный снаружи поворота, повернется на полные градусы, в то время как двигатель, расположенный внутри, рассчитает требуемое расстояние в соответствии с ожидаемым поворотом.

##### Пример

In [ ]:
#!/usr/bin/env python3
import time

from ev3dev2.motor import MoveTank, OUTPUT_A, OUTPUT_B

tank_pair = MoveTank(OUTPUT_A, OUTPUT_B)

print("90 degrees")
tank_pair.on_for_degrees(50, 50, 90)
time.sleep(1)

#### on_for_rotations *(left_speed, right_speed, rotations, brake=True, block=True)*

Вращает двигатели с **левой скоростью** и **правой скоростью** на количество **оборотов**. Скорости могут быть выражены в процентах или в любой реализации значения скорости.

Если левая скорость не равна правой скорости (т.е. робот повернет), двигатель, расположенный снаружи поворота, совершит полный оборот, в то время как двигатель, расположенный внутри, рассчитает требуемое расстояние в соответствии с ожидаемым поворотом.

##### Пример

In [ ]:
#!/usr/bin/env python3
import time

from ev3dev2.motor import MoveTank, OUTPUT_A, OUTPUT_B

tank_pair = MoveTank(OUTPUT_A, OUTPUT_B)

print("3 rotations")
tank_pair.on_for_rotations(50, 50, 3)
time.sleep(1)

#### on_for_seconds *(left_speed, right_speed, seconds, brake=True, block=True)*

Вращает двигатели с **левой скоростью** и **правой скоростью** в течение **секунд**. Скорости могут быть выражены в процентах или в любой реализации значения скорости.

##### Пример

In [ ]:
#!/usr/bin/env python3
import time

from ev3dev2.motor import MoveTank, OUTPUT_A, OUTPUT_B

tank_pair = MoveTank(OUTPUT_A, OUTPUT_B)

print("3 seconds")
tank_pair.on_for_seconds(50, 50, 3)
time.sleep(1)

#### on *(left_speed, right_speed)*

Начинает вращать двигатели в соответствии с **left_speed** и **right_speed** вечно. Скорости могут быть выражены в процентах или в любой реализации значения скорости.

##### Пример

In [ ]:
#!/usr/bin/env python3
import time

from ev3dev2.motor import MoveTank, OUTPUT_A, OUTPUT_B

tank_pair = MoveTank(OUTPUT_A, OUTPUT_B)

print("Forever")
tank_pair.on(50, 50)
time.sleep(10)
tank_pair.off()

#### follow_line _(kp, ki, kd, speed, target_light_intensity=None, follow_left_edge=True, white=60, off_line_count_max=20, sleep_time=0.01, follow_for=<function follow_for_forever>, **kwargs)_ *

Следует по линии, используя PID регулятор.

**kp**, **ki** и **kd** - константы PID.

**speed** - желаемая скорость средней точки робота.

**target_light_intensity** - интенсивность отраженного света, когда датчик цвета
находится на краю линии. Если это значение равно None, мы предполагаем, что датчик цвета находится на краю линии и будет считывать показания, чтобы установить эту переменную.

**follow_left_edge** определяет, следуем ли мы за левым или правым краем линии.

**white** - это отраженная интенсивность света, которая используется для определения того, потеряли ли мы линию.

**off_line_count_max** - это сколько раз подряд в цикле **reflected_light_intensity** должно быть больше **white**, прежде чем мы объявим линию потерянной и вызовем исключение.

**sleep_time** - это сколько секунд мы ожидаем при каждом прохождении цикла. Это нужно для того, чтобы дать роботу возможность отреагировать на новые настройки двигателя. Это должно быть что-то небольшое, например 0,01 (10 мс).

**follow_for** вызывается, чтобы определить, следует ли нам продолжать следовать линии или остановиться. Эта функция будет передана *self* (текущему объекту *Move Tank*). Поддерживаемые в настоящее время опции: **follow_for_forever**, **follow_for_ms**.

****kwargs** будут переданы в функцию **follow_for**.

##### Пример

In [ ]:
#!/usr/bin/env python3
from ev3dev2.motor import (OUTPUT_A, OUTPUT_B, LineFollowErrorTooFast,
                           MoveTank, SpeedPercent, follow_for_ms)
from ev3dev2.sensor.lego import ColorSensor

tank = MoveTank(OUTPUT_A, OUTPUT_B)
tank.cs = ColorSensor()

try:
    # Следуем линии на протяжении 4500 мс
    tank.follow_line(
        kp=11.3, ki=0.05, kd=3.2,
        speed=SpeedPercent(30),
        follow_for=follow_for_ms,
        ms=4500
    )
except LineFollowErrorTooFast:
    tank.stop()
    raise

#### follow_gyro_angle _(kp, ki, kd, speed, target_angle=0, sleep_time=0.01, follow_for=<function follow_for_forever>, **kwargs)_ *

Следует по углу гироскопа, используя PID регулятор.

**kp**, **ki** и **kd** - константы PID.

**speed** - желаемая скорость средней точки робота.

**target_angle** - это угол, который мы хотим сохранить.

**sleep_time** - это сколько секунд мы ожидаем при каждом прохождении цикла. Это нужно для того, чтобы дать роботу возможность отреагировать на новые настройки двигателя. Это должно быть что-то небольшое, например 0,01 (10 мс).

**follow_for** вызывается, чтобы определить, следует ли нам продолжать следовать желаемому углу или остановиться. Эта функция будет передана *self* (текущему объекту *Move Tank*). Поддерживаемые в настоящее время опции: **follow_for_forever**, **follow_for_ms**.

****kwargs** будут переданы в функцию **follow_for**.

##### Пример

In [ ]:
#!/usr/bin/env python3
from ev3dev2.motor import (OUTPUT_A, OUTPUT_B, FollowGyroAngleErrorTooFast,
                           MoveTank, SpeedPercent, follow_for_ms)
from ev3dev2.sensor.lego import GyroSensor

# Инициализируем моторы
tank = MoveTank(OUTPUT_A, OUTPUT_B)

# Инициализируем гироскоп
tank.gyro = GyroSensor()

# Калибруем гироскоп для устранений дрейфа и установки текущего угла как 0
tank.gyro.calibrate()

try:

    # Следуем заданном углу на протяжении 4500 мс
    tank.follow_gyro_angle(
        kp=11.3, ki=0.05, kd=3.2,
        speed=SpeedPercent(30),
        target_angle=0,
        follow_for=follow_for_ms,
        ms=4500
    )
except FollowGyroAngleErrorTooFast:
    tank.stop()
    raise

#### turn_degrees *(speed, target_angle, brake=True, error_margin=2, sleep_time=0.01)* *

Использует датчик гироскопа для поворота на месте на **target_angle** угол.

**speed** - желаемая скорость средней точки робота.

**target_angle** - это угол, на который мы хотим повернуть.

**brake** - останавливает робота при достижении заданного угла.

**error_margin** - это порог +/- угла, позволяющий контролировать, насколько точным должен быть поворот.

**sleep_time** - это сколько секунд мы ожидаем при каждом прохождении цикла. Это нужно для того, чтобы дать роботу возможность отреагировать на новые настройки двигателя. Это должно быть что-то небольшое, например 0,01 (10 мс).

##### Пример

In [ ]:
from ev3dev2.motor import OUTPUT_A, OUTPUT_B, MoveTank, SpeedPercent
from ev3dev2.sensor.lego import GyroSensor

tank = MoveTank(OUTPUT_A, OUTPUT_B)

# Инициализируем гироскоп
tank.gyro = GyroSensor()

# Калибруем гироскоп для устранений дрейфа и установки текущего угла как 0
tank.gyro.calibrate()

# Поворот на 30 градусов
tank.turn_degrees(
    speed=SpeedPercent(5),
    target_angle=30
)

#### turn_right *(speed, degrees, brake=True, error_margin=2, sleep_time=0.01)* *

Поворачивает робота по часовой стрелке на месте на заданное количество градусов.

#### turn_left *(speed, degrees, brake=True, error_margin=2, sleep_time=0.01)* *

Поворачивает робота против часовой стрелки на месте на заданное количество градусов.

### MoveSteering _(left_motor_port, right_motor_port, desc=None, motor_class=<class 'ev3dev2.motor.LargeMotor'>)_

Управляет двумя двигателями на основе значения **рулевого управления _(steering value)_** и **скорости**. Немного сложнее объяснить значение этих параметров по сравнению с параметрами **MoveTank**. **Значение рулевого управления** определяет траекторию движения робота и может принимать любое значение **в диапазоне от -100 до +100**.

- -100 означает поворот налево на месте;

- -50 означает поворот налево без поворота левого колеса;

- 0 означает движение прямо;

- +50 означает поворот направо без поворота правого колеса;

- +100 означает поворот направо на месте.

Значение скорости относится к скорости более быстрого колеса (иногда колеса могут быть одинаково быстрыми, например, при рулевом управлении = 0). Скорость более медленного колеса рассчитывается автоматически на основе значений рулевого управления и скорости.

Следующая таблица может помочь понять взаимосвязь между **MoveTank** и **MoveSteering**. Он предназначен для того, чтобы показать, какие значения **left_speed** и **right_speed** (параметры **MoveTank**) соответствуют всем возможным значениям **рулевого управления** и значению **скорости**, равному 40 *(speed=40)* (параметры **MoveSteering**).

<img src="images/motors/SteeringValues.png" alt="SteeringValues">

- При всех значениях **рулевого управления** *(от -100 до +100)* всегда имеется по крайней мере один двигатель с частотой вращения, равной значению **скорости** 40.

- При **рулевом управлении=0** отмечаем, что **left_speed** и **right_speed** **равны скорости**, которая равна 40.

- При **рулевом управлении=25** отмечаем, что **left_speed=40** и **right_speed=20**.

- При значении **рулевого управления=50** отмечаем, что **левая скорость=40**, а **правая скорость=0**.

- При значении **рулевого управления=100** отмечаем, что **левая скорость=40**, а **правая скорость=-40**.

#### on_for_rotations *(steering, speed, rotations, brake=True, block=True)*

Вращает двигатели в соответствии с прилагаемым **рулевым управлением**.

Расстояние, которое пройдет каждый двигатель, соответствует правилам *MoveTank.on_for_rotations()*.

##### Пример

In [ ]:
#!/usr/bin/env python3
import time

from ev3dev2.motor import MoveSteering, OUTPUT_A, OUTPUT_B

steering_pair = MoveSteering(OUTPUT_A, OUTPUT_B)

print("3 rotations")
steering_pair.on_for_rotations(0, 50, 3)
time.sleep(1)

#### on_for_degrees *(left_speed, right_speed, degrees, brake=True, block=True)*

Вращает двигатели в соответствии с прилагаемым **рулевым управлением**.

Расстояние, которое пройдет каждый двигатель, соответствует правилам *MoveTank.on_for_degrees()*.

##### Пример

In [ ]:
#!/usr/bin/env python3
import time

from ev3dev2.motor import MoveSteering, OUTPUT_A, OUTPUT_B

steering_pair = MoveSteering(OUTPUT_A, OUTPUT_B)

print("90 degrees")
steering_pair.on_for_degrees(0, 50, 90)
time.sleep(1)

#### on_for_seconds *(left_speed, right_speed, seconds, brake=True, block=True)*

Вращает двигатели в соответствии с прилагаемым **рулевым управлением** в течение заданного количества **секунд**.

##### Пример

In [ ]:
#!/usr/bin/env python3
import time

from ev3dev2.motor import MoveSteering, OUTPUT_A, OUTPUT_B

steering_pair = MoveSteering(OUTPUT_A, OUTPUT_B)

print("3 seconds")
steering_pair.on_for_seconds(50, 50, 3)
time.sleep(1)

#### on *(left_speed, right_speed)*

Начинает вращать двигатели в соответствии с **рулевым управлением** и **скоростью** вечно.

##### Пример

In [ ]:
#!/usr/bin/env python3
import time

from ev3dev2.motor import MoveSteering, OUTPUT_A, OUTPUT_B

steering_pair = MoveSteering(OUTPUT_A, OUTPUT_B)

print("Forever")
steering_pair.on(0, 50)
time.sleep(10)
steering_pair.off()

## Задание

1. Проехать по периметру стола-стенда *(244-122 мм)*.
2. Перед заездом указать количество кругов, которое проедет робот, а так же скорость, используя кнопки.
3. На пути появляются препядствия на заранее известном месте и в течение маршрута не меняются. К предыдущему заданию добавляется указание с какой стороны будет препядствие (справа/слева).

### Примечание

Задание может меняться по усмотрению преподавателя.